### 무신사 썸네일 이미지 크롤링
    
    최초 작성일 : 20/03/03
    작성자 : 김은주
    
    작성내용 : 무신사 썸네일 이미지 크롤링
               
               뀨뀨꺄꺄
               -> 
               
    수정내용 
        20/03/03
               뀨뀨꺄꺄
               -> 

In [1]:
import requests, pymysql, math, re, time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# from selenium import webdriver                                  # selenium의 webdriver을 임포트 해준다. 
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.keys import Keys                 # selenium 키 값을 이용

import urllib.request

In [ ]:
"""
이미지크롤링 참고
https://rekt77.tistory.com/100
https://geundung.dev/36?category=705611

MySQL 이미지 저장 참고
https://cleancode-ws.tistory.com/102

썸네일
<img src="//image.msscdn.net/images/goods_img/20180824/836981/836981_2_500.jpg" width="500" title="" alt="더블유브이프로젝트(WV PROJECT) 와핑 후드집업 그레이 JJOT7203" id="bigimg" class="plus_cursor" onclick="image_zoom.showLayer('big_box', 'big_img', 'detail_thumb', currentImageIdx);">
"""

In [ ]:
## DB 연결 데이터 
config = {"host":"127.0.0.1", "user":"root", "passwd":"1111", "db":"musinsa_test"}
conn = pymysql.connect(**config)

In [2]:
# 샘플 url
prdt_link = ["https://store.musinsa.com/app/product/detail/836981/0",
             "https://store.musinsa.com/app/product/detail/621582/0"]

def crawl_prdt_img(prdt_link):
    for url in prdt_link:
        html, req = "", requests.get(url)
        if req.status_code == 200 :
            html = req.text
        print("HTTP Status :", req)
        soup = BeautifulSoup(html, 'html.parser')
        
        soup = soup.find("div", class_="product-img")
        #img의 경로를 받아온다
        imgUrl = soup.find("img")["src"]
        
        print(imgUrl)

        #urlretrieve는 다운로드 함수
        urllib.request.urlretrieve(imgUrl, soup.find("img")["alt"]+'.jpg')

In [3]:
crawl_prdt_img(prdt_link)

HTTP Status : <Response [200]>
//image.msscdn.net/images/goods_img/20180824/836981/836981_2_500.jpg


ValueError: unknown url type: '//image.msscdn.net/images/goods_img/20180824/836981/836981_2_500.jpg'

In [ ]:
"""
## 랭킹DB에서 상품링크 중복제외 불러오기
prdt_link = set()
cursor = conn.cursor()

rank_table = ['TOP']
# ['ALL', 'TOP', 'OUTER', 'ONEPIECE', 'BOTTOM', 'SKIRT', 'BAG', 'SNEAKER', 'SHOE', 'WATCH', 'HAT', 'SPORT', 'UNDER', 'GLSS', 'ACC', 'DIGITAL', 'ART', 'COSMETIC', 'PET', 'BOOK']

for table in rank_table:
    sql = "SELECT PRDT_LINK FROM RANK_"+table+";"
    cursor.execute(sql)

    for i in cursor.fetchall():
        prdt_link.add(str(i[0]))
    
############################################################################ << 상품정보

# 샘플 url
prdt_link = ["https://store.musinsa.com/app/product/detail/836981/0",
             "https://store.musinsa.com/app/product/detail/621582/0"]

for url in list(prdt_link):
    html, req = "", requests.get(url)
    if req.status_code == 200 :
        html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    print("HTTP Status :", req)
    
    code = []
    product = []
    delivery = []
    price = []
    hashtegs = ''
    priceCheck = [] # 할인상품 / 비할인상품 확인용
    
    
## CODE_PRDT&SUB
    code.append(["prdt_code", url[45:len(url)-2]])
    code.append(["sub_code", soup.find("p", class_="item_categories").select("a")[1].get('href')[17:].strip()])

# ## Product Info  
    ## BRD_NAME_KOR 가져오기
    product.append(["brd_name", soup.select("p>strong>a")[0].get("href")[22:].strip()])

    ## Product Info / SATISFACTION) 가져오기
    product.append(["satisfaction", soup.find("div", class_="estimate-stats").find("span", class_="rate")\
                    .text.replace(" 만족", "").strip()])

    for p in soup.find_all("p", class_="product_article_contents"):            
    ## PRDT_SEX 가져오기
        if p.find("span", class_="txt_gender") != None:
            product.append(["p_sex", p.select("span>span")[0].text.strip()])
            
    ## REVIEW_CNT 가져오기
        if p.find("a", class_="link_type") != None:
            product.append(["review_cnt", p.find("a", class_="link_type")\
                            .text[:len(p.find("a", class_="link_type").text)-1].replace(",", "").strip()])

    ## PRDT_HASH 가져오기
        if len(p.find_all("a", class_="listItem")) != 0:
            for a in p.find_all("a", class_="listItem"):
                hashtegs += a.text.replace(" ", "").strip()
            product.append(["p_hash", hashtegs])

## Price Info가격정보
        if p.find("span", class_="product_article_price") != None:
            if p.find("span", class_="product_article_price").find("del") != None: ## 할인 상품 정상가 가져오기
                priceCheck.append(p.find("span", class_="product_article_price").find("del").text.replace(",", "").strip())
            else:                                                                  ## 비할인 정상가 & 할인가 가져오기
                priceCheck.append(p.find("span", class_="product_article_price").text.replace(",", "").strip())

    if len(priceCheck) == 1: # 비할인 상품
        price.append(["fixed_price", priceCheck[0]])
        price.append(["sale_price", priceCheck[0]])
    else: # 할인 상품
        price.append(["fixed_price", priceCheck[0]])
        price.append(["sale_price", priceCheck[1]])

# Delivery Info / AVG_DELDATE, LOGISTICS 가져오기
    for p in soup.find_all("p", class_="product_article_contents del"):
        delivery.append(["avg_deldate", p.text[:p.text.index('일')].strip()])
        delivery.append(["logistics", p.text[p.text.index('일')+4:len(p.text)-23].strip()])


    dict_code = dict(code)
    dict_product = dict(product)
    dict_delivery = dict(delivery)
    dict_price = dict(price)
    print(dict_code, dict_product, dict_delivery, dict_price)
    
## DB 적재
    sql = "INSERT INTO PRDT_INFO(BRD_NAME_KOR, PRDT_CODE, PRDT_SEX, PRDT_HASH, AVG_DELDATE, "\
         + "LOGISTICS, SUB_CODE, FIXED_PRICE, SALE_PRICE, REVIEW_CNT, SATISFACTION) VALUES("\
         + "'" + dict_product.get("brd_name", '오류') +"'" +  ","\
         + "'" + dict_code.get("prdt_code", '오류') +"'" +  ","\
         + "'" + dict_product.get("p_sex", '오류') +"'" +  ","\
         + "'" + dict_product.get("p_hash", '오류') +"'" +  ","\
         + "'" + dict_delivery.get("avg_deldate", '오류') +"'" +  ","\
         + "'" + dict_delivery.get("logistics", '오류') +"'" +  ","\
         + "'" + dict_code.get("sub_code", '오류') +"'" +  ","\
         + "'" + dict_price.get("fixed_price", '오류') +"'" +  ","\
         + "'" + dict_price.get("sale_price", '오류') +"'" +  ","\
         + "'" + dict_product.get("review_cnt", '오류') +"'" +  ","\
         + "'" + dict_product.get("satisfaction", '오류') +"'" + ");"
    cursor.execute(sql)
    conn.commit()
    
    print(dict_product.get("brd_name"), "완료")

############################################################################ 상품정보 >>

conn.close()
"""